# Zadanie 16

In [1]:
import numpy as np

In [133]:
def mean_absolute_error(ground_truth, identified):
    assert len(ground_truth) == len(identified), "Arrays must have the same length"
    mae = np.mean(np.abs(ground_truth - identified))

    return mae


def root_mean_squared_error(ground_truth, identified):
    assert len(ground_truth) == len(identified), "Arrays must have the same length"
    rmse = np.sqrt(np.mean((ground_truth - identified) ** 2))

    return rmse


def max_absolute_error(ground_truth, identified):
    assert len(ground_truth) == len(identified), "Arrays must have the same length"
    max_error = np.max(np.abs(ground_truth - identified))

    return max_error


def sca_evolve(plut: np.ndarray, x: np.ndarray) -> np.ndarray:
    probs = plut[7 - (np.roll(x, 1) * 4 + x * 2 + np.roll(x, -1))]
    rnd = np.random.random(size=probs.shape)
    return (rnd < probs).astype(np.int8)


def sca_evolve_spacetime(plut: np.ndarray, initial_conf: np.ndarray, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(sca_evolve(plut, rows[-1]))
    return np.stack(rows)


def generate_config(config_length: int) -> np.ndarray:
    return np.random.randint(2, size=config_length)


def generate_plut(length: int, threshold: (int, float) = 0.4, lower: bool = False) -> np.ndarray:
    if lower:
        return np.round(threshold * np.random.random(length), 2)
    return np.round(threshold + (1 - threshold) * np.random.random(length), 2)


def ca_identify(observation: np.ndarray):
    neighborhoods = np.roll(observation, -1, axis=1) + observation * 2 + 4 * np.roll(observation, 1, axis=1)
    return neighborhoods


# Gwóźdź programu

In [ ]:
def identify_plut(spacetime_diagram: np.ndarray) -> np.ndarray:
    counter = np.zeros((8, 2), dtype=int)

    identified = ca_identify(spacetime_diagram)[:-1]
    diagram = spacetime_diagram[1:]

    np.add.at(counter, (identified.flatten(), diagram.flatten()), 1)
    identified_plut = counter[:, 1] / np.sum(counter, axis=1)

    return identified_plut[::-1]

In [136]:
plut = generate_plut(8, 0, lower=False)
config = generate_config(8)

diagram = sca_evolve_spacetime(plut, config, 1000)

identified_plut = identify_plut(diagram)

print(f'Original pLUT   : {plut}')
print(f'Identified pLUT : {np.round(identified_plut, 2)}')

Original pLUT   : [0.14 0.63 0.5  0.66 0.64 0.18 0.91 0.65]
Identified pLUT : [0.13 0.63 0.48 0.68 0.63 0.18 0.92 0.66]


In [141]:
# Calculate Mean Absolute Error
mae = mean_absolute_error(plut, identified_plut)
print("Mean Absolute Error:", mae)

# Calculate Root Mean Squared Error
rmse = root_mean_squared_error(plut, identified_plut)
print("Root Mean Squared Error:", rmse)

# Calculate Maximum Absolute Error
max_error = max_absolute_error(plut, identified_plut)
print("Maximum Absolute Error:", max_error)

Mean Absolute Error: 0.010288193572050026
Root Mean Squared Error: 0.01155112009976636
Maximum Absolute Error: 0.017241379310344807


### Możemy ustawiać jakie jest minimalne prawdopodobieństwo w plucie np. z przedziału [0.1, 1], [0.7, 1]. Będę sprawdzał czy ma to jakiś wpływ na błędy oszacowania pluta. Wylosuję 200 losowych konfiguracji o długości 50 i 20 plutów z dla każdego przedziału prawdopodobieństwa. Wyliczę różne średnie błędy.

In [160]:
configs = [generate_config(50) for _ in range(200)]
thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
pluts = [generate_plut(8, threshold, lower=False) for threshold in thresholds for _ in range(20)]

In [161]:
mean_mea = []
mean_rmse = []
mean_maxae = []

for i, plut in enumerate(pluts):
    for config in configs:
        meas = []
        rmses = []
        maxaes = []

        identified_plut = identify_plut(sca_evolve_spacetime(plut, config, 100))
        meas.append(mean_absolute_error(plut, identified_plut))
        rmses.append(root_mean_squared_error(plut, identified_plut))
        maxaes.append(max_absolute_error(plut, identified_plut))

    mean_mea.append(np.mean(meas))
    mean_rmse.append(np.mean(rmses))
    mean_maxae.append(np.mean(maxaes))

C:\Users\kubal\AppData\Local\Temp\ipykernel_13384\2049770395.py:36: RuntimeWarning: invalid value encountered in divide
  identified_plut = counter[:, 1] / np.sum(counter, axis=1)


In [162]:
print('MEA:')
grouped_array = np.array(mean_mea).reshape(-1, 20)
mean_values = np.mean(grouped_array, axis=1)
print(np.round(mean_values, 4))

print('RMSE:')
grouped_array = np.array(mean_rmse).reshape(-1, 20)
mean_values = np.mean(grouped_array, axis=1)
print(np.round(mean_values, 4))

print('MaxEA:')
grouped_array = np.array(mean_maxae).reshape(-1, 20)
mean_values = np.mean(grouped_array, axis=1)
print(np.round(mean_values, 4))


MEA:
[0.0157 0.016  0.0144 0.0155 0.0192 0.02   0.0221 0.0242 0.0288 0.0251]
RMSE:
[0.0208 0.0201 0.0188 0.0206 0.0249 0.0277 0.032  0.0353 0.0419 0.0375]
MaxEA:
[0.0435 0.0372 0.0381 0.042  0.0505 0.0614 0.0718 0.0779 0.0926 0.0868]


## Widzimy że zawężając przedział prawdopodobieństwa otrzymujemy większe błędy. Zapewne z powodu tego, że nasz spacetime diagram dąży do samych jedynek i trudnej jest zliczać zera, a przez to trudniej oszacować prawdopodobieństwa.  